In [52]:
from gurobipy import *

import numpy as np

In [53]:
node_no = {
    'A':0, 'B':1, 'C':2, 'D':3, 'E':4, 'F':5, 'G':6, 'H':7
}

In [56]:
with open('distances.txt', 'r') as file:
    distances = [list(map(float, line.split())) for line in file]
distances

[[0.0, 1.0, 2.0, 1.0, 1.0, 2.0, 2.0, 1.0],
 [1.0, 0.0, 1.0, 1.0, 2.0, 3.0, 1.0, 3.0],
 [2.0, 1.0, 0.0, 1.0, 2.0, 2.0, 3.0, 2.0],
 [1.0, 1.0, 1.0, 0.0, 1.0, 2.0, 1.0, 2.0],
 [1.0, 2.0, 2.0, 1.0, 0.0, 1.0, 3.0, 2.0],
 [2.0, 3.0, 2.0, 2.0, 1.0, 0.0, 1.0, 2.0],
 [2.0, 1.0, 3.0, 1.0, 3.0, 1.0, 0.0, 1.0],
 [1.0, 3.0, 2.0, 2.0, 2.0, 2.0, 1.0, 0.0]]

In [57]:
paths = []
with open('paths.txt', 'r') as file:
    for line in file:
        path = line.strip().split(': ')[1].split('-')
        start = node_no[path[0]]
        end = node_no[path[-1]]
        total_dist = 0
        for el in range(len(path)-1):
            dest = path[el + 1]
            source = path[el] 
            total_dist += distances[node_no[source]][node_no[dest]]
        paths.append({'total_dist':total_dist, 'start':start, 'end': end})
        
paths

[{'total_dist': 7.0, 'start': 0, 'end': 1},
 {'total_dist': 3.0, 'start': 1, 'end': 0},
 {'total_dist': 4.0, 'start': 2, 'end': 3},
 {'total_dist': 9.0, 'start': 3, 'end': 2},
 {'total_dist': 3.0, 'start': 4, 'end': 2},
 {'total_dist': 2.0, 'start': 7, 'end': 5},
 {'total_dist': 5.0, 'start': 0, 'end': 4},
 {'total_dist': 5.0, 'start': 1, 'end': 2},
 {'total_dist': 4.0, 'start': 2, 'end': 7},
 {'total_dist': 9.0, 'start': 5, 'end': 4},
 {'total_dist': 2.0, 'start': 0, 'end': 2},
 {'total_dist': 2.0, 'start': 0, 'end': 5},
 {'total_dist': 5.0, 'start': 1, 'end': 3},
 {'total_dist': 5.0, 'start': 6, 'end': 4},
 {'total_dist': 2.0, 'start': 1, 'end': 6}]

In [58]:
distances_to_depots = []
with open('depot_node_distances.txt', 'r') as file:
    for line in file:
        distances_to_depots.append([int(x) for x in line.strip().split(': ')[1].split('-')])


distances_to_depots


[[1, 1, 1, 2, 3, 2, 1, 1], [3, 2, 1, 1, 1, 1, 1, 2]]

In [59]:
depot_no = {
    'X':0, 'Y':1
}

In [60]:
odd_even = {}

for i in range(len(paths)):
        for j in range(2):
            time = 20
            #for X depot
            time -= distances_to_depots[j][paths[i]['start']]
            tour = 0
            while(time > paths[i]['total_dist']):
                time -= paths[i]['total_dist']
                tour += 1
            if tour % 2 == 0: #even number of tours
                time -= distances_to_depots[j][paths[i]['start']]
                if time >= 0:
                    #print("success! begin and end nodes are same for route")
                    odd_even[i, j] = 1
                else:
                    time += paths[i]['total_dist']
                    tour -= 1
                    odd_even[i, j] = 0

                    #print("you need to take one less tour, sorry, ODDIZED")
            else: #odd number of tours
                time -= distances_to_depots[j][paths[i]['end']]
                if time >= 0:
                    #print("success! begin and end nodes are not same for route")
                    odd_even[i, j] = 0
                else:
                    time = time + paths[i]['total_dist']
                    tour -= 1
                    odd_even[i, j] = 1
                    #print("you need to take one less tour, sorry , EVENIZED")
            
odd_even

{(0, 0): 1,
 (0, 1): 1,
 (1, 0): 1,
 (1, 1): 0,
 (2, 0): 1,
 (2, 1): 1,
 (3, 0): 0,
 (3, 1): 1,
 (4, 0): 0,
 (4, 1): 1,
 (5, 0): 1,
 (5, 1): 1,
 (6, 0): 0,
 (6, 1): 0,
 (7, 0): 0,
 (7, 1): 0,
 (8, 0): 1,
 (8, 1): 1,
 (9, 0): 0,
 (9, 1): 1,
 (10, 0): 0,
 (10, 1): 0,
 (11, 0): 1,
 (11, 1): 0,
 (12, 0): 0,
 (12, 1): 0,
 (13, 0): 0,
 (13, 1): 0,
 (14, 0): 0,
 (14, 1): 1}

In [61]:
X = {}
model = Model('Part1_model')

for i in range(1,16):
    for j in ['X','Y']:
        for k in range(1,9):
            X[i,j,k] = model.addVar(vtype = GRB.BINARY,name=f'x_{i}_{j}_{k}')

#Constraints
#Trains assigned to deport j starts from node k
for i in range(1,16):
    model.addConstr(quicksum(X[i,j,k] for k in range(1,9) for j in ['X','Y']) == 1)
     
        
#A route cannot be used more than 3 trains
for j in ['X','Y']:
    for k in range(1,9):
        model.addConstr(quicksum(X[i,j,k] for i in range(1,16)) <= 3)
        
#Each depot shpuld have at least 5 trains assigned to it
for j in ['X','Y']:
    model.addConstr(quicksum(X[i,j,k] for k in range(1,9) for i in range(1,16)) >= 5)
        
#Every train should have only one starting node
#for k in range(1,9):
#    model.addConstr(quicksum(X[i,j,k] for i in range(1,16) for j in ['X','Y']) == 1)


objective_func = quicksum((1 + odd_even[i - 1, depot_no[j]]) * X[i,j,k] * distances_to_depots[depot_no[j]][paths[i-1]['start']] + 
                          (1 - odd_even[i - 1, depot_no[j]]) * X[i,j,k] * distances_to_depots[depot_no[j]][paths[i-1]['end']] 
                          for i in range(1,16) for j in ['X', 'Y'] for k in range(1, 9))
model.setObjective(objective_func, GRB.MINIMIZE)

model.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 22.4.0 22E252)

CPU model: Apple M2 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 33 rows, 240 columns and 720 nonzeros
Model fingerprint: 0xf964deeb
Variable types: 0 continuous, 240 integer (240 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 6e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+00]
Found heuristic solution: objective 49.0000000
Presolve time: 0.00s
Presolved: 33 rows, 240 columns, 720 nonzeros
Variable types: 0 continuous, 240 integer (240 binary)
Found heuristic solution: objective 33.0000000

Root relaxation: cutoff, 31 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0     cutoff    0        33.00000   33.00000  0.00%   

In [62]:
for j in ['X', 'Y']:
    assigned_trains = [i for k in range(1, 9) for i in range(1, 16) if X[i, j, k].X > 0.5]
    print(f"Depot {j} assigned trains: {assigned_trains}")

Depot X assigned trains: [1, 11, 15, 2, 6, 9, 12, 8, 13]
Depot Y assigned trains: [7, 10, 14, 4, 5, 3]
